# Game Script
This script is used to train communities of senders & receivers to play the discrimination game. Metrics about training development and validation performance are saved for later visualization and analysis. In addition, an evaluation dataset is produced to allow a detailed analysis of the emerging language and agent's performance on the test data.

----------------------------------------

## Load Packages

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import egg.core as core
from egg.core import Callback, Interaction, PrintValidationEvents
from torchvision import models, datasets, transforms
import numpy as np
import random
import matplotlib.pyplot as plt
import csv
from IPython.display import clear_output
import time
import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Preparing the Game Datasets
The game datasets are stored as a txt file in which each row corresponds to the data needed for playing the game once and evaluating the agent's performance on it. Before they can be used for training, validation or testing, the datasets need to be converted to a format that is compatible with EGG and PyTorch.

In [2]:
class EGG_Dataset(Dataset):
    '''
    A dataset that is read in from a txt file and is compatible with the EGG framework and PyTorch's DataLoader.
    
    Attributes:
        frame -- The dataset
        
    Methods:
        __init__(game_data_path, test_or_val) -- Constructor, initializes a dataset object
        get_n_features() -- Returns the dimension of an image embedding
        __len__() -- Returns the lenght of the dataset
        __getitem__(idx) -- Returns an element of the dataset by the given index idx
    '''
    def __init__(self, game_data_path, test = False):
        '''
        Constructor, initializes an EGG_Dataset object, i.e. a dataset that is compatible with both EGG
        and PyTorch's DataLoader.
        
        Parameter:
            game_data_path -- The path to the game data (train, val, or test) for which a dataset is to be produced
            test -- Boolean that indicates whether or not the dataset will be used for testing
        '''
        self.frame = []
        with open(game_data_path, 'r') as game_data:
            for i,line in enumerate(game_data):
                # splits row into list with an element for each embedding and the target idx (at last position)
                raw_info = line.split(",")
                # stores the embeddings in nested list index_vectors and casts values to int
                # , e.g. [[feat1, feat2, feat3], [feat1, feat2, feat3]]
                embeddings = list([list(map(float, x.split())) for x in raw_info[:-2]])
                # imagenet class of the target image
                target_class = int(raw_info[-2])
                # index of the target embedding
                target_idx = torch.tensor(int(raw_info[-1]))
                # the target embedding, which is the sender input
                target_embedding = torch.FloatTensor(embeddings[target_idx])
                # constructing the receiver input, i.e. storing target and distractor embeddings in one tensor
                target_and_distractors = []
                for embedding in embeddings:
                    target_and_distractors = np.concatenate((target_and_distractors, embedding))
                target_and_distractors = torch.FloatTensor(target_and_distractors).view(len(embeddings), -1)
                # for training data: storing sender input, target_index, and receiver input in data frame
                if test == False:
                    self.frame.append((target_embedding, target_idx, target_and_distractors))
                # for test or val data: additionally sotre the imagenet class of the target
                else:
                    self.frame.append((target_embedding, target_idx, target_and_distractors, target_class))
        
    def get_n_features(self):
        'Returns the dimension of the image embeddings'
        return self.frame[0][0].size(0)

    def __len__(self):
        'Returns the length of the dataset'
        return len(self.frame)

    def __getitem__(self, idx):
        'Returns an element of the dataset by index'
        return self.frame[idx]

In [3]:
train_dataset = EGG_Dataset('./Data/Game_Data/train.txt')

In [4]:
val_dataset = EGG_Dataset('./Data/Game_Data/val.txt')

## Implement Core Agent Designs
EGG-agents are made up of two elements: a core and a wrapper. The wrappers are provided by EGG and implement the connection between sender and receiver, i.e. the message production, transition and receival. The cores define remaining agent behavior. For the senders, the core maps the sender input to an input for its wrapper. For the receivers, the core maps the output of its wrapper to an output about the game, i.e. the receivers decision.

In [5]:
class Sender(nn.Module):
    '''
    The core of any sender agent. It takes the embedding of the target image as input and produces an initial
    hidden state for the message producing GRU (that will be initialized via an EGG-Wrapper below).
    
    Attributes:
        fc1 -- The only layer of the sender's core
    
    Methods:
        __init__(n_hidden, n_features) -- Constructor, initializes the sender's core
        forward(x, _aux_input) -- Performs a forward pass through the sender's core 
    '''
    def __init__(self, n_hidden, n_features):
        '''
        Constructor, initializes the sender's core.
        
        Parameter:
            n_features -- The input size, i.e. the number of elements in an image embedding
            n_hidden -- The output size, i.e. the size of the hidden states in the message producing GRU
            
        Output:
            None
        '''
        super(Sender, self).__init__()
        self.fc1 = nn.Linear(n_features, n_hidden)

    def forward(self, x, _aux_input):
        '''Performs a forward pass through the sender's core, i.e. maps the target img embedding x to 
        the initial hidden state of the GRU and returns this mapping'''
        return self.fc1(x)
        return self.fc1(x).tanh()
        #return self.fc1(x).LeakyReLu
        # here, it might make sense to add a non-linearity, such as tanh



class Receiver(nn.Module):
    '''
    The core of any receiver agent. It takes two inputs: a message embedding from its wrapper GRU and the image
    embeddings of the target and distractors. Target and distractor embeddings are mapped to the same dimension
    as the message embedding and then computes the dot product between the message embedding and each of the mapped
    image embeddings. The resulting list of dot products is interpreted as a non-normalized probability distribution
    over possible target positions, i.e. the highest dot product was computed using the mapped embedding of the target image.
    
    Attributes:
        fc1 -- The only layer of the receiver's core
        
    Methods:
        __init__(n_features, n_hidden) -- Constructor, initializes a receiver's core
        forward(x, _input, _aux_input) -- Performs a forward pass through the receiver's core
    '''
    def __init__(self, n_features, n_hidden):
        '''
        Constructor, initalizes the sender's core.
        
        Parameter:
            n_features -- The input sizes of the target and distractor embeddings
            n_hidden -- The size of the wrapper-GRU's hidden state
            
        Output:
            None
        '''
        super(Receiver, self).__init__()
        self.fc1 = nn.Linear(n_features, n_hidden)

    def forward(self, x, _input, _aux_input):
        '''
        Performs a forward pass through the receiver's core, i.e. maps all embeddings of target and distractor
        images to the dimension of the wrapper-GRU's message embedding and then computes and returns the dot
        products between all target/distractor mappings and the message embedding.
        
        Parameter:
            x -- The message embedding produced by the wrapper-GRU
            _input -- The embeddings of target and distractor images
        
        Output:
            dots -- A list of dot products between mapped image embeddings and the message embedding, the element
                    with the highest dot product acts as the receiver's prediction of the target position
        '''
        # the rationale for the non-linearity here is that the RNN output (x) will also be the outcome of a non-linearity
        embedded_input = self.fc1(_input).tanh()
        dots = torch.matmul(embedded_input, torch.unsqueeze(x, dim=-1))
        #return dots.squeeze()
        return F.softmax(dots.squeeze(), dim = 1)

## Set Parameters

In [6]:
# game parameter
comm_size = 4
sender_comm_size = comm_size
receiver_comm_size = comm_size
vocab_size = 10
max_len = 5

# agent parameter
sender_hidden = 10
receiver_hidden = 35
sender_embedding = 5
receiver_embedding = 5
sender_cell = 'gru'
receiver_cell = 'gru'
n_features = train_dataset.get_n_features()

# training parameter, defined on egg level for optimizer call in game initialization
patience = 50
opts = core.init(params=['--lr=1e-4',
                         '--batch_size=128',
                         '--optimizer=adam'])

# setting a PyTorch seed
seed = 5
torch.manual_seed(seed)

## Load Datasets

In [7]:
train_loader = DataLoader(train_dataset, batch_size=opts.batch_size, shuffle=True, num_workers=0, drop_last = True, pin_memory=False)
val_loader = DataLoader(val_dataset, batch_size=opts.batch_size, shuffle=True, num_workers=0, drop_last = True, pin_memory=False)

## Define Loss Function

In [8]:
def loss(_sender_input, _message, _receiver_input, receiver_output, labels, _aux_input):
    '''
    The loss function used for optimizing the game agents. EGG requires the loss function to have all these
    parameters even if they are not used for the loss computation.
    
    Parameter (all parameters have on value for each element in the batch):
        _sender_input -- The input to the sender (an image embedding), not used for my loss computation
        _message -- The message produced by the sender, not used for my loss computation
        _receiver_input -- The input to the receiver (target + distractors), not used for my loss computation
        receiver_output -- The output of the receiver
        labels -- The indexes of the target images
        
    Output:
        loss, {'acc': acc}
            loss -- The cross entropy between the receiver output and the labels
            acc -- The success rates (0 or 1) of all samples in the batch
    '''
    global batch_accs, batch_losses, validate
    # in the discriminative case, accuracy is computed by comparing the index with highest score in Receiver output (a distribution of unnormalized
    # probabilities over target poisitions) and the corresponding label read from input, indicating the ground-truth position of the target
    acc = (receiver_output.argmax(dim=1) == labels.to(device)).detach().float()
    # similarly, the loss computes cross-entropy between the Receiver-produced target-position probability distribution and the labels
    loss = F.cross_entropy(receiver_output, labels.to(device), reduction="none")
    if validate:
        batch_accs.append(torch.mean(acc).item())
        batch_losses.append(torch.mean(loss).item())
        
    
    return loss, {"acc": acc}

----------------------------------------

# Discrimination Game with Agent Communities

### Initializing the Agent Communities
First, I set the size of our sender/receiver community. Then, I initialize the respective number of agents, apply the correct EGG wrappers, and store them in lists.

##### Initializing the Agents, applying EGG wrappers and storing agents in lists

In [9]:
sender_comm = []
receiver_comm = []

for i in range(sender_comm_size):
    current_sender = Sender(n_hidden=sender_hidden, n_features=n_features)
    sender_comm.append(core.RnnSenderReinforce(
                            current_sender,
                            vocab_size=vocab_size,
                            embed_dim=sender_embedding,
                            hidden_size=sender_hidden,
                            cell=sender_cell,
                            max_len=max_len,
                            ).to(device)
    )

for i in range(receiver_comm_size):
    current_receiver = Receiver(n_features=n_features, n_hidden=receiver_hidden)
    receiver_comm.append(core.RnnReceiverDeterministic(
                            current_receiver,
                            vocab_size=vocab_size,
                            embed_dim=receiver_embedding,
                            hidden_size=receiver_hidden,
                            cell=receiver_cell,
                            ).to(device)
    )

### Initialize all possible Games
For each possible pair of sender & receiver, we need to initialize a game.

In [10]:
games_callbacks_optimizers = []

for sender_idx,sender in enumerate(sender_comm):
    for receiver_idx,receiver in enumerate(receiver_comm):
        current_game = core.SenderReceiverRnnReinforce(
                            sender,
                            receiver,
                            loss,
                            sender_entropy_coeff=0,
                            receiver_entropy_coeff=0,
                            )
        current_callbacks = []
        current_optimizer = core.build_optimizer(current_game.parameters())
        games_callbacks_optimizers.append((sender_idx, receiver_idx, current_game, current_callbacks, current_optimizer))

### Intialize the Respective Trainers

In [11]:
trainers = []

for (sender_idx, receiver_idx, current_game, current_callbacks, current_optimizer) in games_callbacks_optimizers:
    trainers.append((
        sender_idx, 
        receiver_idx, 
        core.Trainer(
            game=current_game,
            optimizer=current_optimizer,
            train_data=train_loader,
            callbacks=current_callbacks+ [core.ConsoleLogger(print_train_loss=True, as_json=True)],
            )
    ))

### Functions for a Train & Validation Steps and the Visualization of Training Progress

In [12]:
'''
def train(egg_trainer):
    global batch_accs, batch_losses
    batch_accs = []
    batch_losses = []
    egg_trainer.train(n_epochs = 1)

    return (np.array(batch_accs).mean(), np.array(batch_losses).mean())
'''
def check_performance(dataset):
    '''
    Checks the communities performance on the provided dataset. To do so, a pair is sampled from the community
    for each batch in the dataset. That pair's loss and accuracy are computed and stored.
    
    Parameter:
        dataset -- The dataset for which the communities performance is to be tested.
        
    Output:
        (accuracy, loss)
            accuracy -- The average accuracy achieved by the sampled pairs on the dataset's batches.
            loss -- The average loss achieved by the sampled pairs on the dataset's batches.
    '''
    global batch_accs, batch_losses
    batch_accs = []
    batch_losses = []
    for batch in dataset:
        try:
            (sender_inputs, target_idxs, receiver_inputs, target_classes) = batch
        except ValueError:
            (sender_inputs, target_idxs, receiver_inputs) = batch
            
        sender_idx = random.randrange(0, len(sender_comm))
        receiver_idx = random.randrange(0, len(receiver_comm))
        with torch.no_grad():
            sender = sender_comm[sender_idx]
            receiver = receiver_comm[receiver_idx]

            messages, __sen_logprobs, __sen_entropies = sender(sender_inputs.to(device))
            rec_outputs, __rec_logrobs, __rec_entropies = receiver(messages, receiver_inputs.to(device))
            loss_fn_output = loss(sender_inputs, messages, receiver_inputs, rec_outputs, target_idxs, None)
        
    return (np.array(batch_accs, dtype=float).mean(), np.array(batch_losses, dtype=float).mean())

def visualize(train_accs, train_losses, val_accs, val_losses, val_epochs):
    '''
    Plots the communities performance on the train and on the validation dataset.
    
    Parameter:
        train_accs -- The accuracies on the training data achieved by the community for all completed epochs
        train_losses -- The losses on the training data achieved by the community for all completed epochs
        val_accs -- The accuracies on the validation data achieved by the community for all completed epochs
        val_losses -- The losses on the validation data achieved by the community for all completed epochs
        
    Output:
        None
        Prints the plots in the console
    '''
    global total_nr_epochs
    clear_output(wait = True)
    plt.figure(figsize = (15,7))
    line1, = plt.plot(range(0,len(train_losses)),train_losses)
    line2, = plt.plot(range(0,len(val_losses)),val_losses)
    plt.xlabel("Epoch")
    plt.xlim(0,total_nr_epochs)
    plt.ylabel("Loss")
    plt.legend((line1,line2),("training","validation"))
    plt.grid(True)
    plt.show()

    plt.figure(figsize = (15,7))
    line1, = plt.plot(range(0,len(train_accs)),train_accs)
    line2, = plt.plot(range(0,len(val_accs)),val_accs)
    plt.xlabel("Epoch")
    plt.xlim(0,total_nr_epochs)
    plt.ylabel("Accuracy")
    plt.ylim(0,1.05)
    plt.yticks(np.arange(0, 1.05, 0.05))
    plt.legend((line1,line2),("training","validation"))
    plt.grid(True)
    plt.show()

def check_early_stopping(current_epoch, epoch_val_acc, val_accs, comm_size, patience):
    '''
    Checks whether the epoch with the best validation accuracy was less thant patience*comm_size*2 epochs ago.
    If yes, then early stopping stays disabled. If it was longer ago, early stopping will be enabled.
    
    Parameter:
        current_epoch -- The number of the current epoch
        epoch_val_acc -- The validation accuracy of the current epoch
        val_accs -- All validation accuracies computed so far
        comm_size -- The community size
        patience -- The patience variable
    
    Output:
        Boolean for enabling early stopping or not.
    '''
    global best_epoch
    if epoch_val_acc > max(val_accs[:-1]):
        best_epoch = current_epoch
        return False
    elif (current_epoch-best_epoch) > (comm_size*comm_size*patience):
        return True

def early_stop(val_losses, comm_size, patience):
    '''
    Called when early stopping is enabled and checks whether to stop training now. It decides to stop training
    if the current validation loss is a minimum.
    
    Parameter:
        val_losses -- All validation losses computed so far
    
    Output:
        Boolean for stopping training right now
    '''
    if val_losses[-1] == min(val_losses[-(comm_size*patience):]):
        return True
    else:
        return False

### Sample a Trainer and Let the Respective Pair Play/Train

In [13]:
total_time_start = time.time()

avg_epochs_per_pair = 200
total_nr_epochs = avg_epochs_per_pair * (sender_comm_size*receiver_comm_size)
visualize_interval = 20 * comm_size

epochs_per_pair = np.zeros((sender_comm_size, receiver_comm_size))
val_epochs = []
train_accs = []
train_losses = []
val_accs = []
val_losses = []
best_epoch = 0

if seed == 1:
    validate = True
else:
    validate = False
    
# recording the untrained performance once before starting to train
if validate:
    initial_train_acc, initial_train_loss = check_performance(train_loader)
    initial_val_acc, initial_val_loss = check_performance(val_loader)
    train_accs.append(initial_train_acc)
    train_losses.append(initial_train_loss)
    val_accs.append(initial_val_acc)
    val_losses.append(initial_val_loss)
    val_epochs.append(0)

for i in range(total_nr_epochs):
    #trainer_idxs = random.sample(range(0,len(trainers)), k = len(trainers))
    #epoch_start = time.time()

    sampled_idx = random.randrange(0, len(trainers))
    (sender_idx, receiver_idx, current_trainer) = trainers[sampled_idx]
    epochs_per_pair[sender_idx][receiver_idx] += 1
    #agent_pairs.append((sender_idx, receiver_idx))
    print('Epoch', i+1 ,'; Currently playing: Sender ', sender_idx+1, ' and Receiver ', receiver_idx+1)
    current_trainer.train(n_epochs = 1)
    
    if validate:
        epoch_train_acc, epoch_train_loss = check_performance(train_loader)
        train_accs.append(epoch_train_acc)
        train_losses.append(epoch_train_loss)
    
        epoch_val_acc, epoch_val_loss = check_performance(val_loader)
        val_accs.append(epoch_val_acc)
        val_losses.append(epoch_val_loss)


        val_epochs.append(i+1)
        
        if (i+1)%visualize_interval == 0:
            visualize(train_accs, train_losses, val_accs, val_losses, val_epochs)
            print('Currently best epoch: ', best_epoch, ' with val acc: ', val_accs[best_epoch])
            print('Lates val acc:', val_accs[-1])
    
total_time_end = time.time()
total_time = total_time_end - total_time_start

Epoch 1 ; Currently playing: Sender  4  and Receiver  4
{"loss": 1.1783522367477417, "acc": 0.3873225450515747, "sender_entropy": 1.6767492294311523, "receiver_entropy": 0.0, "length": 4.242067337036133, "mode": "train", "epoch": 1}
Epoch 2 ; Currently playing: Sender  3  and Receiver  4
{"loss": 1.272673487663269, "acc": 0.5400832891464233, "sender_entropy": 1.2534394264221191, "receiver_entropy": 0.0, "length": 5.400345802307129, "mode": "train", "epoch": 1}
Epoch 3 ; Currently playing: Sender  4  and Receiver  4
{"loss": 1.0210257768630981, "acc": 0.7401259541511536, "sender_entropy": 0.2672814726829529, "receiver_entropy": 0.0, "length": 5.961898326873779, "mode": "train", "epoch": 1}
Epoch 4 ; Currently playing: Sender  3  and Receiver  4
{"loss": 1.1252872943878174, "acc": 0.7693077325820923, "sender_entropy": 0.5630168914794922, "receiver_entropy": 0.0, "length": 5.98364782333374, "mode": "train", "epoch": 1}
Epoch 5 ; Currently playing: Sender  4  and Receiver  3
{"loss": 0.891

Epoch 36 ; Currently playing: Sender  3  and Receiver  3
{"loss": 0.6963340044021606, "acc": 0.8757338523864746, "sender_entropy": 0.08406128734350204, "receiver_entropy": 0.0, "length": 5.999953269958496, "mode": "train", "epoch": 1}
Epoch 37 ; Currently playing: Sender  4  and Receiver  3
{"loss": 0.7077031135559082, "acc": 0.829879641532898, "sender_entropy": 0.03553646802902222, "receiver_entropy": 0.0, "length": 5.9998931884765625, "mode": "train", "epoch": 1}
Epoch 38 ; Currently playing: Sender  1  and Receiver  3
{"loss": 0.7181260585784912, "acc": 0.8800637722015381, "sender_entropy": 0.2152581512928009, "receiver_entropy": 0.0, "length": 4.697287082672119, "mode": "train", "epoch": 1}
Epoch 39 ; Currently playing: Sender  3  and Receiver  1
{"loss": 0.7051917910575867, "acc": 0.8733921051025391, "sender_entropy": 0.08376841247081757, "receiver_entropy": 0.0, "length": 5.999899864196777, "mode": "train", "epoch": 1}
Epoch 40 ; Currently playing: Sender  2  and Receiver  4
{"lo

{"loss": 0.6905713677406311, "acc": 0.875420331954956, "sender_entropy": 0.08934161067008972, "receiver_entropy": 0.0, "length": 5.99998664855957, "mode": "train", "epoch": 1}
Epoch 72 ; Currently playing: Sender  1  and Receiver  1
{"loss": 0.7320957779884338, "acc": 0.9075909852981567, "sender_entropy": 0.1679733693599701, "receiver_entropy": 0.0, "length": 5.089820384979248, "mode": "train", "epoch": 1}
Epoch 73 ; Currently playing: Sender  1  and Receiver  2
{"loss": 0.7018200755119324, "acc": 0.9071773886680603, "sender_entropy": 0.16569198668003082, "receiver_entropy": 0.0, "length": 4.929720878601074, "mode": "train", "epoch": 1}
Epoch 74 ; Currently playing: Sender  4  and Receiver  3
{"loss": 0.7044194936752319, "acc": 0.858000636100769, "sender_entropy": 0.10120262950658798, "receiver_entropy": 0.0, "length": 5.999939918518066, "mode": "train", "epoch": 1}
Epoch 75 ; Currently playing: Sender  1  and Receiver  1
{"loss": 0.7160163521766663, "acc": 0.9162841439247131, "sender_

{"loss": 0.7201026082038879, "acc": 0.8908718228340149, "sender_entropy": 0.12890000641345978, "receiver_entropy": 0.0, "length": 6.0, "mode": "train", "epoch": 1}
Epoch 107 ; Currently playing: Sender  1  and Receiver  2
{"loss": 0.6668527126312256, "acc": 0.9209809899330139, "sender_entropy": 0.1493769884109497, "receiver_entropy": 0.0, "length": 5.013810157775879, "mode": "train", "epoch": 1}
Epoch 108 ; Currently playing: Sender  2  and Receiver  2
{"loss": 0.6660892963409424, "acc": 0.8981840014457703, "sender_entropy": 0.03355254977941513, "receiver_entropy": 0.0, "length": 5.999993324279785, "mode": "train", "epoch": 1}
Epoch 109 ; Currently playing: Sender  4  and Receiver  1
{"loss": 0.6812325119972229, "acc": 0.8847139477729797, "sender_entropy": 0.0846828892827034, "receiver_entropy": 0.0, "length": 5.999993324279785, "mode": "train", "epoch": 1}
Epoch 110 ; Currently playing: Sender  3  and Receiver  1
{"loss": 0.6600475311279297, "acc": 0.9148697853088379, "sender_entropy"

{"loss": 0.6800729036331177, "acc": 0.89251309633255, "sender_entropy": 0.08196967840194702, "receiver_entropy": 0.0, "length": 6.0, "mode": "train", "epoch": 1}
Epoch 142 ; Currently playing: Sender  3  and Receiver  1
{"loss": 0.6557365655899048, "acc": 0.9240566492080688, "sender_entropy": 0.10500317811965942, "receiver_entropy": 0.0, "length": 5.999993324279785, "mode": "train", "epoch": 1}
Epoch 143 ; Currently playing: Sender  1  and Receiver  4
{"loss": 0.861358106136322, "acc": 0.7566449642181396, "sender_entropy": 0.13771283626556396, "receiver_entropy": 0.0, "length": 5.044379711151123, "mode": "train", "epoch": 1}
Epoch 144 ; Currently playing: Sender  4  and Receiver  2
{"loss": 0.6747703552246094, "acc": 0.9023003578186035, "sender_entropy": 0.07759501785039902, "receiver_entropy": 0.0, "length": 5.99998664855957, "mode": "train", "epoch": 1}
Epoch 145 ; Currently playing: Sender  2  and Receiver  2
{"loss": 0.6455398797988892, "acc": 0.9120609760284424, "sender_entropy": 

...

Epoch 3048 ; Currently playing: Sender  4  and Receiver  4
{"loss": 0.6067891716957092, "acc": 0.972065806388855, "sender_entropy": 0.04750356078147888, "receiver_entropy": 0.0, "length": 5.999853134155273, "mode": "train", "epoch": 1}
Epoch 3049 ; Currently playing: Sender  1  and Receiver  4
{"loss": 0.610235333442688, "acc": 0.9738337993621826, "sender_entropy": 0.0787389948964119, "receiver_entropy": 0.0, "length": 5.45169734954834, "mode": "train", "epoch": 1}
Epoch 3050 ; Currently playing: Sender  1  and Receiver  3
{"loss": 0.5950484275817871, "acc": 0.9807723164558411, "sender_entropy": 0.07304844260215759, "receiver_entropy": 0.0, "length": 5.439107894897461, "mode": "train", "epoch": 1}
Epoch 3051 ; Currently playing: Sender  4  and Receiver  4
{"loss": 0.605503499507904, "acc": 0.9743741750717163, "sender_entropy": 0.04854685440659523, "receiver_entropy": 0.0, "length": 5.999826431274414, "mode": "train", "epoch": 1}
Epoch 3052 ; Currently playing: Sender  3  and Receiver  

Epoch 3083 ; Currently playing: Sender  3  and Receiver  2
{"loss": 0.5932955741882324, "acc": 0.9823201298713684, "sender_entropy": 0.04530101642012596, "receiver_entropy": 0.0, "length": 5.999973297119141, "mode": "train", "epoch": 1}
Epoch 3084 ; Currently playing: Sender  3  and Receiver  2
{"loss": 0.5931239128112793, "acc": 0.9823268055915833, "sender_entropy": 0.04557197540998459, "receiver_entropy": 0.0, "length": 5.999933242797852, "mode": "train", "epoch": 1}
Epoch 3085 ; Currently playing: Sender  3  and Receiver  4
{"loss": 0.6062737703323364, "acc": 0.9726062417030334, "sender_entropy": 0.04684250429272652, "receiver_entropy": 0.0, "length": 5.999973297119141, "mode": "train", "epoch": 1}
Epoch 3086 ; Currently playing: Sender  3  and Receiver  4
{"loss": 0.604745090007782, "acc": 0.9749413132667542, "sender_entropy": 0.04647524654865265, "receiver_entropy": 0.0, "length": 5.999919891357422, "mode": "train", "epoch": 1}
Epoch 3087 ; Currently playing: Sender  2  and Receiv

Epoch 3119 ; Currently playing: Sender  3  and Receiver  1
{"loss": 0.592608630657196, "acc": 0.9817196726799011, "sender_entropy": 0.047428347170352936, "receiver_entropy": 0.0, "length": 5.999913215637207, "mode": "train", "epoch": 1}
Epoch 3120 ; Currently playing: Sender  2  and Receiver  2
{"loss": 0.6029835939407349, "acc": 0.9605839252471924, "sender_entropy": 0.015306267887353897, "receiver_entropy": 0.0, "length": 6.0, "mode": "train", "epoch": 1}
Epoch 3121 ; Currently playing: Sender  1  and Receiver  4
{"loss": 0.6097655892372131, "acc": 0.9726929664611816, "sender_entropy": 0.06581821292638779, "receiver_entropy": 0.0, "length": 5.446820259094238, "mode": "train", "epoch": 1}
Epoch 3122 ; Currently playing: Sender  3  and Receiver  1
{"loss": 0.5928305983543396, "acc": 0.9816929697990417, "sender_entropy": 0.047767531126737595, "receiver_entropy": 0.0, "length": 5.999899864196777, "mode": "train", "epoch": 1}
Epoch 3123 ; Currently playing: Sender  1  and Receiver  2
{"los

{"loss": 0.5926690101623535, "acc": 0.9835277199745178, "sender_entropy": 0.046531010419130325, "receiver_entropy": 0.0, "length": 5.999946594238281, "mode": "train", "epoch": 1}
Epoch 3155 ; Currently playing: Sender  2  and Receiver  1
{"loss": 0.6001132130622864, "acc": 0.9639464020729065, "sender_entropy": 0.01458536833524704, "receiver_entropy": 0.0, "length": 6.0, "mode": "train", "epoch": 1}
Epoch 3156 ; Currently playing: Sender  2  and Receiver  4
{"loss": 0.6103638410568237, "acc": 0.9576683640480042, "sender_entropy": 0.014802944846451283, "receiver_entropy": 0.0, "length": 6.0, "mode": "train", "epoch": 1}
Epoch 3157 ; Currently playing: Sender  2  and Receiver  1
{"loss": 0.599013090133667, "acc": 0.964633584022522, "sender_entropy": 0.01438919361680746, "receiver_entropy": 0.0, "length": 5.9999799728393555, "mode": "train", "epoch": 1}
Epoch 3158 ; Currently playing: Sender  2  and Receiver  4
{"loss": 0.6098818182945251, "acc": 0.9579352736473083, "sender_entropy": 0.014

{"loss": 0.608587384223938, "acc": 0.9754816889762878, "sender_entropy": 0.06740312278270721, "receiver_entropy": 0.0, "length": 5.465074062347412, "mode": "train", "epoch": 1}
Epoch 3190 ; Currently playing: Sender  3  and Receiver  4
{"loss": 0.604691743850708, "acc": 0.9756618142127991, "sender_entropy": 0.0451357439160347, "receiver_entropy": 0.0, "length": 5.999953269958496, "mode": "train", "epoch": 1}
Epoch 3191 ; Currently playing: Sender  3  and Receiver  4
{"loss": 0.6055346727371216, "acc": 0.9738538265228271, "sender_entropy": 0.043874725699424744, "receiver_entropy": 0.0, "length": 5.999946594238281, "mode": "train", "epoch": 1}
Epoch 3192 ; Currently playing: Sender  2  and Receiver  4
{"loss": 0.6104693412780762, "acc": 0.9567610621452332, "sender_entropy": 0.014670068398118019, "receiver_entropy": 0.0, "length": 6.0, "mode": "train", "epoch": 1}
Epoch 3193 ; Currently playing: Sender  3  and Receiver  3
{"loss": 0.5926874279975891, "acc": 0.9795847535133362, "sender_ent

In [14]:
if validate == False:
    validate = True
    final_train_acc, final_train_loss = check_performance(train_loader)
    final_val_acc, final_val_loss = check_performance(val_loader)
    train_accs.append(final_train_acc)
    train_losses.append(final_train_loss)
    val_accs.append(final_val_acc)
    val_losses.append(final_val_loss)
    validate = False
print('Comm-Size:', comm_size)
print('Seed: ', seed)
print('Batch-Size: ', opts.batch_size)
print('Learning Rate: ', opts.lr)
print('Sender Hidden: ', sender_hidden)
print('Receiver Hidden: ', receiver_hidden)
print('Sender Embedding: ', sender_embedding)
print('Receiver Embedding: ', receiver_embedding, '\n')
print('Final accs (train, val): ',train_accs[-1], val_accs[-1])
print('Best accs (train, val): ', max(train_accs), max(val_accs))
print('Final losses (train, val): ',train_losses[-1], val_losses[-1])
print('Min Losses (train, val): ', min(train_losses), min(val_losses))
print('Epochs per pair: ', epochs_per_pair.flatten())
print('Total time needed (minutes): ', total_time/60)

Comm-Size: 4
Seed:  5
Batch-Size:  128
Learning Rate:  0.0001
Sender Hidden:  10
Receiver Hidden:  35
Sender Embedding:  5
Receiver Embedding:  5 

Final accs (train, val):  0.9632125320239112 0.9208400106837606
Best accs (train, val):  0.9632125320239112 0.9208400106837606
Final losses (train, val):  0.605025183075417 0.6341690762430174
Min Losses (train, val):  0.605025183075417 0.6341690762430174
Epochs per pair:  [174. 218. 197. 202. 202. 190. 199. 181. 226. 204. 188. 216. 192. 207.
 212. 192.]
Total time needed (minutes):  1018.655518468221


In [15]:
if validate:
    visualize(train_accs, train_losses, val_accs, val_losses, val_epochs)

### Storing Training Metrics
Storing training epochs per agent, accuarcy scores with pair indexes, and batch sizes.

In [16]:
# to store: val_epochs, train_accs, train_losses, val_accs, val_losses, epochs_per_pair

# storing the number of training epochs per pair   
with open('./Data/Training_Data/Comm_Size_'+str(comm_size)+'/Seed_'+str(seed)+'/epochs_per_pair.txt', 'w') as f:
    epochs_per_pair_list = []
    for array in epochs_per_pair:
        epochs_per_pair_list.append(list(array))
    
    json.dump(epochs_per_pair_list, f)

# storing the epochs in which validation was performed
with open('./Data/Training_Data/Comm_Size_'+str(comm_size)+'/Seed_'+str(seed)+'/val_epochs.txt', 'w') as f:
    json.dump(val_epochs, f)
    
# storing the accuracies on the train set
with open('./Data/Training_Data/Comm_Size_'+str(comm_size)+'/Seed_'+str(seed)+'/train_accs.txt', 'w') as f:
    json.dump(train_accs, f)

# storing the losses on the train set
with open('./Data/Training_Data/Comm_Size_'+str(comm_size)+'/Seed_'+str(seed)+'/train_losses.txt', 'w') as f:
    json.dump(train_losses, f)

# storing the accuracies on the validation set
with open('./Data/Training_Data/Comm_Size_'+str(comm_size)+'/Seed_'+str(seed)+'/val_accs.txt', 'w') as f:
    json.dump(val_accs, f)
    
# storing the losses on the validation set
with open('./Data/Training_Data/Comm_Size_'+str(comm_size)+'/Seed_'+str(seed)+'/val_losses.txt', 'w') as f:
    json.dump(val_losses, f)

### Saving the Agents

In [17]:
for i in range(comm_size):
    sender_file_name = 'sender_'+str(i+1)+'.pt'
    sender_path = './Agents/Comm_Size_'+ str(comm_size) +'/Seed_'+str(seed)+'/Senders/'+sender_file_name
    torch.save(sender_comm[i].state_dict(), sender_path)
    receiver_file_name = 'receiver_'+str(i+1)+'.pt'
    receiver_path = './Agents/Comm_Size_'+ str(comm_size) +'/Seed_'+str(seed)+'/Receivers/'+receiver_file_name
    torch.save(receiver_comm[i].state_dict(), receiver_path)